# AI medium challenge writeup

Importing the libraries

In [18]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

import numpy as np
from PIL import Image
import requests
from io import BytesIO
import base64

from scipy.optimize import differential_evolution
from scipy.optimize import OptimizeResult

Here we set some properties about the server that we want to attack and the image size, which can be seen from the /get_image endpoint.

In [19]:
# Define the target URL. This is the URL of the server that we want to attack
target_url = "http://localhost:8000"
get_image_url = target_url + "/get_image"
submit_image_url = target_url + "/classify_image"
image_size = 64
flag = ""

This function will be used to alter the image by changing the pixel value at the given location.
It will take the image to tamer and a "pixel" variable that contains both the location and the altered color of the image pixel.

In [20]:
def alter_image(image, pixel):
    pixel = pixel.astype(int)
    altered_image = np.copy(image)
    # Change the pixel value at the given location
    altered_image[pixel[0]][pixel[1]][0] = pixel[2]
    altered_image[pixel[0]][pixel[1]][1] = pixel[3]
    altered_image[pixel[0]][pixel[1]][2] = pixel[4]
    
    return altered_image

Here we perform the evaluation of our tampered image. 
We will send the image to the server and get the probability that what we are taking in the office is a cat.
If the response is 200, we will print all the response since the flag is hidden in there.

In [21]:
def probability_of_being_a_cat(image, image_id):
    global flag
    img_str = base64.b64encode(image[0])
    response = requests.post(submit_image_url, json={'image': img_str.decode('UTF-8'), 'id': image_id})
    
    if response.status_code == 200:
        print(response.json())
        flag = response.json()["detail"]["message"]

    try:
        probability = float(response.json()["detail"]["probability_of_being_a_cat"])
    except:
        raise Exception("Error in response: " + str(response.json()))

    return probability

Those functions are used by the differential evolution algorithm:
- `predict_fn` is used to predict the probability of the image being a cat and is used as fitness function for the individuals of out DE algorithm. The algorithm will try to minimize the value returned by this function, trying to make the image less likely to be a cat and more likely to be a dog consequently.
- `callback_fn` is used to stop the algorithm when the image is classified as a dog (i.e. the probability of being a cat is less than 0.5).

In [22]:
def predict_fn(altered_pixel, args):
    # Alter the image by the given amount
    image = alter_image(args[0], altered_pixel)
    return probability_of_being_a_cat([image], id)

def callback_fn(intermediate_result: OptimizeResult):
    return intermediate_result["fun"] < 0.5

Here starts the real attack: First we get the image from the server, parsing the image and the session id for submitting the image later in the process.

In [23]:
# make request to get the image
response = requests.get(get_image_url)

response_json = response.json()

# load the image and resize it
id = response_json["id"]
image = Image.open(BytesIO(base64.b64decode(response_json["image"])))
image = image.resize((image_size, image_size))
image = np.array([image]).astype(np.float32)

We will use the differential evolution algorithm to find the pixel that we need to alter in order to make the image classified as a dog.
The parameters of the algorithm are:
- `predict_fn`: the function that we want to minimize
- `bounds`: the bounds of the pixel that we want to alter: the first two values are the x and y coordinates of the pixel, the other three values are the RGB values of the pixel
- `args`: the arguments that will be passed to the `predict_fn` function. In this case, the image that we want to alter
- `popsize`: the number of individuals in the population
- `tol` and `atol`: the relative tolerance for the stopping criterion: we set them to 0 to disable the relative tolerance and use our custom stopping criterion in the `callback_fn` function
- `maxiter`: the maximum number of iterations
- `callback`: the function that will be called at each iteration: we use it to stop the algorithm when the image is classified as a dog

In [ ]:
# Generate the bounds for the pixel that we need to alter
bounds = [(0,image_size), (0,image_size), (0,255), (0,255), (0,255)]

maxiter=10

# Run the differential evolution algorithm
attack_result = differential_evolution(
    predict_fn, bounds, args=([image]), popsize=8, tol=0, atol=0, maxiter=maxiter, callback=callback_fn
)
print(attack_result)
print(flag)

The DE algorithm will run until the image is classified as a dog or the maximum number of iterations is reached. If the image is classified as a dog, the algorithm will stop and print the result and the flag.

Just for fun, here is our amazing dog that we finally took in the office:

In [25]:
# Alter the image with the pixel found by the differential evolution algorithm
altered_image = alter_image(image[0], attack_result.x)
altered_image = Image.fromarray(altered_image.astype(np.uint8))
# Display the image
altered_image.show()